In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from numpy import loadtxt
from xgboost import  XGBClassifier
from xgboost import plot_importance
import xgboost
from sklearn import metrics
import matplotlib.pyplot as plt
import platform
from matplotlib import font_manager, rc

plt.rcParams['axes.unicode_minus'] = False
if platform.system() == 'Darwin':    # 맥
    rc( 'font', family='AppleGothic' )
elif platform.system() == 'Windows': # 윈도우
    fontPath = 'c:/Windows/Fonts/malgun.ttf'
    fontName = font_manager.FontProperties( fname=fontPath ).get_name()
    rc( 'font', family=fontName )
else:
    print('알수없는 시스템. 미적용')
    
pd.set_option('display.expand_frame_repr', False)
import warnings
warnings.filterwarnings(action='ignore')

# Selection


In [3]:
final_dataset = pd.read_excel('./han.xlsx')

col = ['순매수수량(외국인계)(20일합산)(주)_1일변화율',
 '순매수수량(기관/외국인계)(60일합산)(주)_1일변화율',
 '순매수수량(기관계)(주)_1일변화율',
 '수익률 (1주)(%)_1일변화율',
 '변동성 (5일)_2일변화율',
 '순매수수량(외국인계)(20일합산)(주)_2일변화율',
 'PER(IFRS-연결)_2일변화율',
 '순매수수량(외국인계)(주)_2일변화율',
 '거래량 (5일 평균)(주)_2일변화율','수익률(%)(2)','Name']

data =  final_dataset[col]

In [4]:
def data_sp(data):
    com_name_set = list(set(data.Name))
    
    train_set = pd.DataFrame()
    test_set = pd.DataFrame()
    
    for i in range(len(com_name_set)):
        sub_set = data[ data.Name == com_name_set[i]]
        train_set = train_set.append(sub_set[:-60])
        test_set = test_set.append(sub_set[-60:])
    return train_set , test_set

In [5]:
train, test = data_sp(data)

In [6]:
print(train.shape,test.shape)

(1289, 11) (60, 11)


### 2진화한 데이터셋 분석
## xgboost

In [7]:
col = ['순매수수량(외국인계)(20일합산)(주)_1일변화율',
 '순매수수량(기관/외국인계)(60일합산)(주)_1일변화율',
 '순매수수량(기관계)(주)_1일변화율',
 '수익률 (1주)(%)_1일변화율',
 '변동성 (5일)_2일변화율',
 '순매수수량(외국인계)(20일합산)(주)_2일변화율',
 'PER(IFRS-연결)_2일변화율',
 '순매수수량(외국인계)(주)_2일변화율',
 '거래량 (5일 평균)(주)_2일변화율']
X_train = train[col]
X_test = test[col]

Y_train = train['수익률(%)(2)']
Y_test = test['수익률(%)(2)']

In [8]:
from sklearn import metrics

In [11]:
acc = []
pre= []
param = []
for i in [4,5]:
    print(i)
    for b in [0.01,0.05, 0.1]:
        for c in [0,1,2]:
            for d in [1,2,3]:
                for f in [0.8,0.9,1]:
                    for g in [1500, 5000]:
                        model = XGBClassifier(max_depth=i, 
                                              learning_rate=b,
                                              n_estimators = g,
                                              gamma = c,
                                              min_child_weight=d,
                                              colsample_bytree= f,
                                              n_jobs=-1)

                        model.fit(X_train ,Y_train)
                        a = model.predict(X_test)
                        t = metrics.confusion_matrix(Y_test,a)

                        acc.append(metrics.accuracy_score(a,Y_test))
                        pre.append(t[1,1]/t[:,1].sum())
                        param.append(model)

result = pd.concat([pd.DataFrame(acc),pd.DataFrame(pre),pd.DataFrame(param)],axis=1)
result.columns = ['acc','pre','paremeter']

4
5


In [10]:
result.to_excel('./tuning_result1111.xlsx',encoding = 'utf-8')

NameError: name 'result' is not defined

## Back testing

In [10]:
backtestdata = pd.read_excel('./act.xlsx')

In [11]:
## keep
X_train = train[col]
X_test = test[col]
X_backtestdata = backtestdata[col]

Y_train = train['수익률(%)(2)']
Y_test = test['수익률(%)(2)']
Y_backtestdata = backtestdata['수익률(%)(2)']

model =XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=1, learning_rate=0.01, max_delta_step=0,
       max_depth=5, min_child_weight=2, missing=None, n_estimators=5000,
       n_jobs=-1, nthread=None, objective='binary:logistic',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)

model.fit(X_train[col] ,Y_train)
a = model.predict(X_test[col])
b = model.predict(X_backtestdata[col])
t = metrics.confusion_matrix(Y_backtestdata,b)
print('val: {x:.5}, acc: {y:.5}, pre:{g:.5}'.format(x = metrics.accuracy_score(a,Y_test), y = metrics.accuracy_score(b,Y_backtestdata),
                                                     g = t[1,1]/t[:,1].sum() ),'\n', metrics.confusion_matrix(Y_backtestdata,b))

val: 0.68333, acc: 0.56522, pre:0.6 
 [[4 6]
 [4 9]]


In [108]:
b

array([1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1,
       1], dtype=int64)

In [127]:
np.array(backtestdata['수익률(%)']*b)

array([ 0.67,  2.94, -0.  ,  0.67,  0.  , -1.05,  1.99,  0.  , -0.65,
       -0.26,  2.63,  0.  ,  1.01, -0.25,  3.14, -0.  , -0.  ,  3.11,
        0.24,  0.  , -0.  , -0.84,  1.45])

In [129]:
(np.array(backtestdata['수익률(%)']*b)+100)/100

array([1.0067, 1.0294, 1.    , 1.0067, 1.    , 0.9895, 1.0199, 1.    ,
       0.9935, 0.9974, 1.0263, 1.    , 1.0101, 0.9975, 1.0314, 1.    ,
       1.    , 1.0311, 1.0024, 1.    , 1.    , 0.9916, 1.0145])

In [138]:
tt = np.array(backtestdata['수익률(%)']*b)


In [142]:
ttt = (tt/100)+1

In [143]:
tttt = ttt[ttt!=1]

In [144]:
tttt

array([1.0067, 1.0294, 1.0067, 0.9895, 1.0199, 0.9935, 0.9974, 1.0263,
       1.0101, 0.9975, 1.0314, 1.0311, 1.0024, 0.9916, 1.0145])

In [148]:
1.0067 * 1.0294 * 1.0067 * 0.9895 * 1.0199* 0.9935* 0.9974* 1.0263* 1.0101* 0.9975* 1.0314* 1.0311* 1.0024*0.9916*1.0145

1.1569219809709768